In [5]:
import math
import random
from sympy import randprime, isprime, Mod

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [6]:
def GenModulus(w):
    n = len(w) // 2
    p = randprime(2 ** n, 2 ** (n+1))
    q = randprime(2 ** n, 2 ** (n+1))
    N = p * q
    return N, p, q

In [7]:
def GenRSA(w):
    n = len(w)
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    d = modinv(e, m)
    return N, e, d, p, q
# N, e - public key
# N, d - private key

def enc(x, N, e):
    return x ** e % N

def dec(c, N, d):
    return c ** d % N

In [8]:
N, e, d, p, q = GenRSA("111111111111111111111")

In [9]:
message = random.randint(2, N)

In [10]:
y = enc(message, N, e)

x = dec(y, N, d)


In [201]:
print(message, y, x, message == x)

87361 90572 87361 True


In [202]:
d_bin = "{0:b}".format(d)
print(d, d_bin)

1709873 110100001011100110001


In [203]:
def fast_pow(c, d, N):
    d_bin = "{0:b}".format(d)
    d_len = len(d_bin)
    reduced = 0
    d_ham = 0
    x = c
    for j in range(1, d_len):
        x, r = mod_reduce(x ** 2, N)
        reduced = reduced + r
        if d_bin[j] == "1":
            x, r = mod_reduce(x * c, N)
            reduced = reduced + r
            d_ham = d_ham + 1
            #print("A")
    return x, reduced, d_ham
    

In [204]:
xx, r, h = fast_pow(y, d, N)
print(xx, r, h)

87361 29 9


In [205]:
def mod_reduce(a, b):
    reduced = 0
    if a >= b:
        a = a % b
        reduced = 1
        #print("\tB")
    return a, reduced

In [206]:
c = random.randint(2, N)
m, r, h = fast_pow(c, d, N)
print(r, h)

29 9


Dostarczamy kryptogramy dwóch "rodzajów":
1. y: $y^3 < N$
2. z: $z^2 < N < z^3$

In [157]:
N ** (1/3)

89.1899742061465

In [158]:
N ** (1/2)

842.3140744401699

In [216]:
y = random.randint(2, math.floor(N ** (1/3)))
m, r, h = fast_pow(y, d, N)
print(r, h)

27 9


In [188]:
z = random.randint(math.floor(N ** (1/3)), math.floor(N ** (1/2)))
m, r, h = fast_pow(z, d, N)
print(r, h)

26 9


In [207]:
def dec_blinded(c, d, N):
    r = random.randint(2, N)
    re = enc(r, N, e)
    dec_1, r, h = fast_pow((c * re) % N, d, N)
    print(r)
    r_inv = modinv(r, N)
    return (dec_1 * r_inv) % N

In [223]:
m = dec_blinded(z, d, N)

29
